In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import json
from functools import partial
import pathlib
import glob
from tqdm.notebook import tqdm
from copy import deepcopy

import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd

In [ ]:
import jax
import jax.numpy as jnp
import equinox as eqx

In [ ]:
from mc2.data_management import MaterialSet, Normalizer, EXPERIMENT_LOGS_ROOT, MODEL_DUMP_ROOT, load_data_into_pandas_df
from mc2.features.features_jax import compute_fe_single
from mc2.models.RNN import GRU
from mc2.models.NODE import HiddenStateNeuralEulerODE
from mc2.models.model_interface import NODEwInterface, RNNwInterface, load_model
from mc2.metrics import evaluate_model, evaluate_model_on_test_set

In [ ]:
def featurize(norm_B_past, norm_H_past, norm_B_future, temperature):
    past_length = norm_B_past.shape[0]
    future_length = norm_B_future.shape[0]

    featurized_B = compute_fe_single(jnp.hstack([norm_B_past, norm_B_future]), n_s=10)

    return featurized_B[past_length:]

In [ ]:
data_dict = load_data_into_pandas_df(material="3C90")
mat_set = MaterialSet.from_pandas_dict(data_dict)
mat_set = mat_set.subsample(1)
train_set, val_set, test_set = mat_set.split_into_train_val_test(
    train_frac=0.7, val_frac=0.15, test_frac=0.15, seed=12
)
# train_set_norm = train_set.normalize(transform_H=True, featurize=featurize)
# normalizer = train_set_norm.normalizer

normalizer = Normalizer(
    B_max=1.0,
    H_max=1.0,
    T_max=1.0,
    norm_fe_max=jnp.ones(5),
    H_transform=lambda h: h,
    H_inverse_transform=lambda h: h,
)

In [ ]:
exp_id = "68bcd70f-c34a-43"

In [ ]:
with open(EXPERIMENT_LOGS_ROOT / "jax_experiments" / f"{exp_id}.json") as f:
    exp_results = json.load(f)
    
# interfaced_model = NODEwInterface(
#     load_model(MODEL_DUMP_ROOT / f"{exp_id}.eqx", model_class=HiddenStateNeuralEulerODE),
#     normalizer=normalizer,
#     featurize=featurize,
# )

interfaced_model = RNNwInterface(
    load_model(MODEL_DUMP_ROOT / f"{exp_id}.eqx", model_class=GRU),
    normalizer=normalizer,
    featurize=featurize,
)

In [ ]:
exp_results

In [ ]:
plt.plot(exp_results["logs"]["loss_trends_train"])
plt.yscale("log")

In [ ]:
plt.plot(exp_results["logs"]["loss_trends_val"])
plt.yscale("log")

In [ ]:
# eval_metrics = evaluate_model_on_test_set(
#     interfaced_model,
#     test_set,
# )
# eval_metrics

# plots:

In [ ]:
from mc2.utils.data_plotting import plot_single_sequence, plot_hysteresis

In [ ]:
for frequency_idx in [0,1,2,3,4,5,6]:
    batch_idx=jnp.array([1,2,3,4])
    print("Frequency in Hz: ",test_set.frequencies[frequency_idx])
    H_pred = interfaced_model(
        B_past=test_set[frequency_idx].B[batch_idx, :20],
        H_past=test_set[frequency_idx].H[batch_idx, :20],
        B_future=test_set[frequency_idx].B[batch_idx, 20:],
        T=test_set[frequency_idx].T[batch_idx]
    )#    f=test_set[frequency_idx].frequency
    H_pred.shape
    for H_p, H, B,T in zip(H_pred, test_set[frequency_idx].H[batch_idx, 20:], test_set[frequency_idx].B[batch_idx, 20:], test_set[frequency_idx].T[batch_idx]):

        fig, axs = plot_single_sequence(B, H, T)
        axs[-1].plot(H_p, label="pred")
        fig.legend()
        plt.show()

In [ ]:
for frequency_idx in [0,1,2,3,4,5,6]:
    batch_idx=jnp.array([1,2,3,4])
    print("Frequency in Hz: ",test_set.frequencies[frequency_idx])
    H_pred = interfaced_model(
        B_past=test_set[frequency_idx].B[batch_idx, :20],
        H_past=test_set[frequency_idx].H[batch_idx, :20],
        B_future=test_set[frequency_idx].B[batch_idx, 20:],
        T=test_set[frequency_idx].T[batch_idx]
    )#    f=test_set[frequency_idx].frequency
    H_pred.shape
    for H_p, H, B, T in zip(H_pred, test_set[frequency_idx].H[batch_idx, 20:], test_set[frequency_idx].B[batch_idx, 20:], test_set[frequency_idx].T[batch_idx]):
        
        print(jnp.mean((H_p - H)**2))
        fig, axs = plot_single_sequence(B[:500], H[:500], T)
        axs[-1].plot(H_p[:500], label="pred")
        fig.legend()
        plt.show()

In [ ]:
data = test_set.normalize(normalizer, True)[2]

H_pred = interfaced_model.normalized_call(
    data.B[:10, :20],
    data.H[:10, :20],
    data.B[:10, 20:],
    data.T[:10]
)

for H_p, H, B in zip(H_pred[:10, :480], data.H[:10, 20:500], data.B[:10, 20:500]):
    print(jnp.mean((H_p - H)**2))
    fig, axs = plot_single_sequence(B, H, jnp.unique(data.T))
    axs[-1].plot(H_p, label="pred")
    fig.legend()
    plt.show()